In [1]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from __future__ import print_function

In [2]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [3]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [4]:
sys.path.append('../Python')
from helper import formFH, paths_input

In [5]:
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential,save_model,load_model
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
import keras.optimizers
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [6]:
keras.__version__

'2.0.4'

In [7]:
trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
trDirI = trDirTIF
teDirI = teDirTIF
trWork, teWork = '../Work/Train', '../Work/Test'
trDirI,teDirI, trWork, teWork

('../Data/train-tif-v2',
 '../Data/test-tif-v2',
 '../Work/Train',
 '../Work/Test')

In [8]:
labels_df = pd.read_csv(trLabels)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [9]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [10]:
# Add onehot features for every label
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
    #labels_df[label].astype(np.int8)
# Display head
#labels_df.head()


In [11]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [12]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
labelList[:6,:]
#labels_df.head()

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [13]:
def formImExt (nf, resize=(32,32), printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                nx = cv.resize(ni,resize)
            if (ni.shape[2]==4) :
                #r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,0],ni[:,:,3]
                r,g,b,n = cv.resize(ni[:,:,2],resize),cv.resize(ni[:,:,1],resize),cv.resize(ni[:,:,0],resize),cv.resize(ni[:,:,3],resize)
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                nx      = np.array([r,g,b,n,dv,dw]).T; 
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

In [36]:
trX, trY, i, size = [],[], 0, len(nameList)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameList[0:size] :
    nf = os.path.join(trDirTIF,nn+".tif");
    nx = formImExt(nf)
    if (nx is not None) :
        #rr=np.save(os.path.join(trWork,nn+".npy"),nx);
        #trX.append(nn+".npy")
        trX.append(nx)
        trY.append(True)
    else : 
        trY.append(False)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
    #print(nn.shape)

print(datetime.datetime.now())
trX = np.array(trX);
trY = labelList[trY];
print(len(nameList),trX.shape,trY.shape)
#trXY=pd.DataFrame(trY); trXY['name']=trX; trXY.head()
#trXY.to_pickle(os.path.join(trWork,"listFiles.pkl"))

2017-05-15 01:24:39.256551 40479 40479
2017-05-15 01:25:33.407633 	 5000 	 train_4999
2017-05-15 01:27:48.406526 	 10000 	 train_9999
2017-05-15 01:30:05.273415 	 15000 	 train_14999
2017-05-15 01:32:21.207122 	 20000 	 train_19999
2017-05-15 01:34:37.987198 	 25000 	 train_24999
2017-05-15 01:36:55.296700 	 30000 	 train_29999
2017-05-15 01:39:12.678141 	 35000 	 train_34999
2017-05-15 01:41:33.065293 	 40000 	 train_39999
2017-05-15 01:41:45.582977
40479 (40479, 32, 32, 6) (40479, 17)


In [45]:
np.save('../Data-Keras/train-model-2D-1-v2-XX.h5',trX)
np.save('../Data-Keras/train-model-2D-1-v2-YY.h5',trY)

In [38]:
import keras.backend as K
def fbeta_pred(y_true, y_pred, beta=2.0, OK1=0.2, eps=0.000001, printOK=False):
    beta2 = beta*beta
    yy_true = K.round(y_true)
    #yy_pred = K.round(y_pred+(0.5-OK1))
    yy_pred = K.round(y_pred)
    tp, tp_fp, fn = K.sum((yy_pred*yy_true)), K.sum(yy_true), K.sum((K.abs(yy_pred*(yy_true-1.0))))
    precision, recall = tp/(tp_fp+eps), tp/(tp+fn+eps) 
    fbeta = (1+beta2)*(precision*recall)/(beta2*precision+recall+eps)
    ##if fbeta>1.0 : fbeta = 1.0;
    if printOK :
        print('ten true ',K.get_value(yy_true))
        #print('ten pred ',y_pred)
        print('ten roun ',K.get_value(yy_pred))
        print(' pre=',K.get_value(precision),' recall=',K.get_value(recall),' tp=',
              K.get_value(tp),' fn=',K.get_value(fn),' tp+fp=',K.get_value(tp_fp))
    return(fbeta)

In [60]:
#
#
#

i0,i1,i2,i3 = trX.shape
rr = trY.shape[1]

model = Sequential()
model.add(BatchNormalization(input_shape=(i1,i2,i3)))

model.add(Convolution2D(64,(5,5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(rr,activation='sigmoid'))

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', #sgd, #"adam", #'rmsprop',
              metrics=[fbeta_pred]) #['accuracy',fbeta_pred]) #['accuracy'])

In [61]:
# badly don't work: fail customs metric # model=load_model('../Data-Keras/basin02-loop-all.h5') ## verify load from v1 version
#model.load_weights('../Data-Keras/train-model-020-1D-2-v2-loop-weights.h5') ## verify load weights from v1 version

In [62]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_11 (Batc (None, 32, 32, 6)         24        
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 28, 28, 64)        9664      
_________________________________________________________________
activation_9 (Activation)    (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_34 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 6, 128)         0         
__________

In [65]:
print('time.begin :',datetime.datetime.now())
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.fit(X, y, validation_split=0.2, callbacks=[early_stopping])
hist = model.fit(trX,trY,
         epochs=25, #10,
         batch_size=256,
         validation_split=0.20,
         callbacks=[early_stopping],
         verbose=2
         )
        #samples_per_epoch=200)
        #validation_data=validation_generator,
        #initial_epoch=start,
        #nb_val_samples=2048
         #)
print('time.end   :',datetime.datetime.now())

time.begin : 2017-05-15 02:34:23.642267
Train on 32383 samples, validate on 8096 samples
Epoch 1/25
9s - loss: 4.5661 - fbeta_pred: 0.8535 - val_loss: 4.5398 - val_fbeta_pred: 0.8664
Epoch 2/25
9s - loss: 4.5487 - fbeta_pred: 0.8555 - val_loss: 4.5361 - val_fbeta_pred: 0.8697
Epoch 3/25
9s - loss: 4.5510 - fbeta_pred: 0.8558 - val_loss: 4.5258 - val_fbeta_pred: 0.8653
Epoch 4/25
9s - loss: 4.5304 - fbeta_pred: 0.8562 - val_loss: 4.5286 - val_fbeta_pred: 0.8677
Epoch 5/25
9s - loss: 4.5295 - fbeta_pred: 0.8588 - val_loss: 4.5280 - val_fbeta_pred: 0.8709
Epoch 6/25
9s - loss: 4.5327 - fbeta_pred: 0.8584 - val_loss: 4.5247 - val_fbeta_pred: 0.8667
Epoch 7/25
9s - loss: 4.5281 - fbeta_pred: 0.8562 - val_loss: 4.5310 - val_fbeta_pred: 0.8723
Epoch 8/25
9s - loss: 4.5247 - fbeta_pred: 0.8592 - val_loss: 4.5148 - val_fbeta_pred: 0.8715
Epoch 9/25
9s - loss: 4.5242 - fbeta_pred: 0.8586 - val_loss: 4.5263 - val_fbeta_pred: 0.8721
Epoch 10/25
9s - loss: 4.5220 - fbeta_pred: 0.8583 - val_loss: 4.

In [43]:
save_model(model,'../Data-Keras/train-model-2D-1-v2-all.h5')
model.save_weights('../Data-Keras/train-model-2D-1-v2-weights.h5')  # save weights after training

In [66]:
print(model.evaluate(trX,trY,verbose=2))
trP = model.predict(trX, batch_size=128)
print(trY.shape)
print(trP.shape)
print('fbeta_score=',fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples'))
print('fbeta_pred =',K.get_value(fbeta_pred(trY.astype(np.float64),trP.astype(np.float64))))

[4.3996954664899777, 0.87945910120361415]
(40479, 17)
(40479, 17)
fbeta_score= 0.893811306979
fbeta_pred = 0.879014958228


In [48]:
print('time.begin :',datetime.datetime.now())
beta_old = 0.0
trP = model.predict(trX, batch_size=512)
beta_old = fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples')
print(datetime.datetime.now(),0,beta_old,model.evaluate(trX,trY,verbose=2))
for i in range(10) :
    model.fit(trX,trY,
             nb_epoch=10,
             batch_size=128,
             validation_split=0.20,
             verbose=0
             )
    trP = model.predict(trX, batch_size=512)
    beta_new = fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples')
    print(datetime.datetime.now(),i,beta_new,"(",beta_old,")",model.evaluate(trX,trY,verbose=2))
    if ((beta_old+0.003)<beta_new) :
        save_model(model,'../Data-Keras/train-model-2D-1-v2-loop-all.h5')
        model.save_weights('../Data-Keras/train-model-2D-1-v2-loop-weights.h5')  # save weights after training
        beta_old = beta_new
print('time.end   :',datetime.datetime.now())

time.begin : 2017-05-15 01:58:40.818427
2017-05-15 01:58:43.986029 0 0.868059261389 [4.6235675547191812, 0.85972495279769423]
2017-05-15 01:59:55.872659 0 0.867757843378 ( 0.868059261389 ) [4.634846947510554, 0.85930002745810685]
2017-05-15 02:01:07.827121 1 0.865095514463 ( 0.868059261389 ) [4.6301586407264432, 0.86307829108238576]
2017-05-15 02:02:20.435411 2 0.872059892074 ( 0.868059261389 ) [4.5869103705371481, 0.86574812870481044]
2017-05-15 02:03:32.269407 3 0.87089557615 ( 0.872059892074 ) [4.592559225000457, 0.86118734666265084]
2017-05-15 02:04:44.437900 4 0.865811295641 ( 0.872059892074 ) [4.6107854238361377, 0.86119799125206031]
2017-05-15 02:05:56.442050 5 0.874604359252 ( 0.872059892074 ) [4.5657333543629939, 0.86629773470431837]
2017-05-15 02:07:08.406273 6 0.874211744714 ( 0.872059892074 ) [4.5562372201987262, 0.86594739223163253]
2017-05-15 02:08:19.845920 7 0.86667917166 ( 0.872059892074 ) [4.580335893008769, 0.86347966977631663]
2017-05-15 02:09:32.147029 8 0.86713749

In [68]:
rr, rrx = [], []; trP = model.predict(trX, batch_size=512)
for i in range(trP.shape[1]) :
    xx = [];
    trYY = trY[:,i].astype(np.float64)
    trPY = trP[:,i]
    for ii in range(100) :
        trPP = (trPY>0.01*ii).astype(np.float64)
        #x = fbeta_score(trY[:,0], np.array(trP[:,0] > 0.1*ii), beta=2, average='samples')
        x = K.get_value(fbeta_pred(trYY,trPP))
        xx.append(x)
    rrr = np.array(xx).argmax();
    rr.append(rrr)
    rrx.append(xx[rrr])
    print(datetime.datetime.now(),i,'-->',rrr,xx[rrr],xx[(rrr-1):(rrr+2)])
    #print(xx);
    #plt.plot(np.array(xx)); plt.show()
trM = np.array(rr)/100.0;

2017-05-15 02:40:32.104666 0 --> 64 0.690668014223 [0.68973956976074102, 0.69066801422279211, 0.69014488461294132]
2017-05-15 02:41:01.256546 1 --> 84 0.981194781817 [0.98114260822874111, 0.98119478181654729, 0.98113834097680852]
2017-05-15 02:41:35.939581 2 --> 67 0.817261244201 [0.81649919256186598, 0.81726124420066348, 0.81720181584075324]
2017-05-15 02:42:16.593282 3 --> 92 0.959007189193 [0.95841925761671165, 0.95900718919257255, 0.95877447330169097]
2017-05-15 02:43:02.890658 4 --> 47 0.789400667392 [0.78829811011011819, 0.78940066739193948, 0.78879631139865403]
2017-05-15 02:43:55.147150 5 --> 42 0.57892221217 [0.57814746592484345, 0.5789222121703973, 0.57528661245391677]
2017-05-15 02:44:53.167942 6 --> 57 0.703711548147 [0.70345935610271726, 0.70371154814742987, 0.70346676816201048]
2017-05-15 02:45:57.825965 7 --> 45 0.566625332681 [0.56444230398960549, 0.56662533268054449, 0.56575468552724928]
2017-05-15 02:47:08.351534 8 --> 21 0.259480731412 [0.24770612421548646, 0.2594807

In [34]:
print(trM,'\n',np.array(labels),'\n',rrx,'\n',(np.array(rrx)).mean())

[ 0.69  0.68  0.67  0.57  0.66  0.59  0.75  0.74  0.85  0.62  0.83  0.63
  0.85  0.83  0.67  0.82  0.74] 
 ['haze' 'primary' 'agriculture' 'clear' 'water' 'habitation' 'road'
 'cultivation' 'slash_burn' 'cloudy' 'partly_cloudy' 'conventional_mine'
 'bare_ground' 'artisinal_mine' 'blooming' 'selective_logging' 'blow_down'] 
 [0.93377240657814409, 0.99279538111936494, 0.95448843120120763, 0.9829248177627492, 0.95320055956514071, 0.94823480289015438, 0.94235649860036452, 0.93367768697755682, 0.94515729024076733, 0.95240455248715195, 0.96759549750785712, 0.96064791465733068, 0.93907893839376577, 0.95574064375533718, 0.9357140562398506, 0.93481351993985629, 0.951219273230275] 
 0.951989545362


In [69]:
print(trM,'\n',np.array(labels),'\n',rrx,'\n',(np.array(rrx)).mean())

[ 0.64  0.84  0.67  0.92  0.47  0.42  0.57  0.45  0.21  0.98  0.6   0.15
  0.33  0.46  0.05  0.18  0.05] 
 ['haze' 'primary' 'agriculture' 'clear' 'water' 'habitation' 'road'
 'cultivation' 'slash_burn' 'cloudy' 'partly_cloudy' 'conventional_mine'
 'bare_ground' 'artisinal_mine' 'blooming' 'selective_logging' 'blow_down'] 
 [0.69066801422279211, 0.98119478181654729, 0.81726124420066348, 0.95900718919257255, 0.78940066739193948, 0.5789222121703973, 0.70371154814742987, 0.56662533268054449, 0.25948073141178507, 0.78421972946080287, 0.91042787787543189, 0.25862037752711825, 0.29865100169393016, 0.75683228670815861, 0.10067083166393269, 0.23247956280989218, 0.176990839533778] 
 0.580303778148


In [70]:
save_model(model,  '../Data-Keras/train-model-020-1D-2-v2-loop-all.h5')
model.save_weights('../Data-Keras/train-model-020-1D-2-v2-loop-weights.h5')  # save weights after training

In [115]:
# Forming output dataset for predicting --> trOX, trOY
del(trX)
del(trY)

NameError: name 'trX' is not defined

In [116]:
nameAsk = os.listdir(teDirI); print(len(nameAsk))
trOX, trOY, i, size = [], [], 0, len(nameAsk)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameAsk[0:size] :
    nf = os.path.join(teDirTIF,nn);
    nx = formImExt(nf)
    if (nx is not None) :
        trOX.append(nx)
        trOY.append(nn)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)

print(datetime.datetime.now())

#assert (size!=len(trOY)), "Wrong files {} != {}".format(size,len(trOY))

61191
2017-05-15 03:41:31.480157 40479 61191
2017-05-15 03:44:42.278867 	 5000 	 test_15649.tif
2017-05-15 03:47:38.148162 	 10000 	 test_32423.tif
2017-05-15 03:50:30.404013 	 15000 	 test_36264.tif
2017-05-15 03:53:15.246494 	 20000 	 test_10894.tif
2017-05-15 03:56:02.300335 	 25000 	 file_9502.tif
2017-05-15 03:58:41.448958 	 30000 	 file_14128.tif
2017-05-15 04:01:16.200243 	 35000 	 file_17389.tif
2017-05-15 04:03:46.526075 	 40000 	 test_19356.tif
2017-05-15 04:06:14.270358 	 45000 	 test_10134.tif
2017-05-15 04:08:39.599717 	 50000 	 test_24243.tif
2017-05-15 04:11:03.067190 	 55000 	 test_7049.tif
2017-05-15 04:13:24.769720 	 60000 	 file_10482.tif
2017-05-15 04:13:56.596995


In [117]:
trOX = np.array(trOX);
trOY = np.array([os.path.splitext(x)[0] for x in trOY]);
print(len(nameAsk),trOX.shape,trOY.shape)

61191 (61191, 32, 32, 6) (61191,)


In [ ]:
# Saving & Loading

In [118]:
np.save('../Data-Keras/test-basin-2D-1-OX-tif-v2.npy',trOX)
np.save('../Data-Keras/test-basin-2D-1-OY-tif-v2.npy',trOY)

In [119]:
trOX = np.load('../Data-Keras/test-basin-2D-1-OX-tif-v2.npy')
trOY = np.load('../Data-Keras/test-basin-2D-1-OY-tif-v2.npy')
print(trOX.shape,trOY.shape)

(61191, 32, 32, 6) (61191,)


In [ ]:
#Forming result

In [120]:
trP = model.predict(trOX, batch_size=512); 
trP.shape, trP[-1]

((61191, 17), array([  8.55755119e-04,   9.99668956e-01,   2.44944282e-02,
          9.97647941e-01,   2.16208771e-02,   6.90812198e-03,
          2.93815713e-02,   1.97501872e-02,   4.83314099e-04,
          8.18360932e-05,   6.91951578e-03,   5.09938673e-06,
          1.69814972e-03,   4.79749997e-06,   5.87242134e-02,
          3.13019678e-02,   6.51171524e-03], dtype=float32))

In [121]:
wr = [labels.index(i) for i in weather_labels];
trM[np.array(wr)] = 2.0
print(wr,'\n',labels,'\n',trM.tolist())

[3, 10, 0, 9] 
 ['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down'] 
 [2.0, 0.84, 0.67, 2.0, 0.47, 0.42, 0.57, 0.45, 0.21, 2.0, 2.0, 0.15, 0.33, 0.46, 0.05, 0.18, 0.05]


In [122]:
#trP = model.predict(trX, batch_size=512); trP=K.get_value(trP)
res = []

for i in range(trP.shape[0]) :
    trPP = [weather_labels[trP[i,wr].argmax()]] + [labels[ii] for ii in range(len(labels)) if (trP[i,ii]>trM[ii])];
    pp   = ' '.join(trPP)
    ##if (pp=="") : print(trY[i])
    res.append([trOY[i],pp])

res.sort(cmp=lambda x,y: cmp(int(x[0].partition('_')[2]),int(y[0].partition('_')[2])) if (x[0].partition('_')[0]==y[0].partition('_')[0]) else cmp(y[0].partition('_')[0],x[0].partition('_')[0]))
#print(res[4:8],'\n',res[-4:])
print(res[0],'\n',res[-1],'\n',trM,trP[0])

['test_0', 'clear primary'] 
 ['file_20521', 'clear primary agriculture road'] 
 [ 2.    0.84  0.67  2.    0.47  0.42  0.57  0.45  0.21  2.    2.    0.15
  0.33  0.46  0.05  0.18  0.05] [  5.39046887e-04   9.99957800e-01   1.14646405e-02   9.97690916e-01
   6.60496298e-03   1.89214305e-03   4.40627150e-03   5.26360050e-03
   6.80364537e-05   1.28222273e-05   5.01615694e-03   1.58569605e-07
   1.30850254e-04   8.27282207e-08   9.06800292e-03   1.74645672e-03
   6.92547299e-04]


In [123]:
print(labels)
print(trM.tolist())
np.round(trP[4:11,:])

['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']
[2.0, 0.84, 0.67, 2.0, 0.47, 0.42, 0.57, 0.45, 0.21, 2.0, 2.0, 0.15, 0.33, 0.46, 0.05, 0.18, 0.05]


array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]], dtype=float32)

In [124]:
rrr=pd.DataFrame(res,columns=['image_name','tags']); rrr.head(); 
suffixDT = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S'); print(suffixDT)
rrr.to_csv('../Result/vss'+suffixDT+'.csv',index=False);

2017-05-15-04-20-07
